In [49]:
!pip install pandas matplotlib numpy seaborn scikit-learn

Defaulting to user installation because normal site-packages is not writeable
distutils: /home/yls1/.local/lib/python3.9/site-packages
sysconfig: /home/yls1/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [50]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns
import sklearn
import requests
import json

In [51]:
# mcolors.BASE_COLORS #these colors can be called with a single character

# mcolors.TABLEAU_COLORS #the default color cycle colors

# mcolors.CSS4_COLORS #named colors also recognized in css

# mcolors.XKCD_COLORS #named colors from the xkcd survey

In [52]:
def Alt_data_check_clean(uni) : 
    # Read in Altmetric Data
    data = pd.read_csv(f"../Data/Alt/Altmetric_{uni}.csv", encoding = "utf-8")
    print(f"Read the {uni} Alt local data file!")
    
    data_extra = data.copy(deep=True)
    # data_extra = data_extra.dropna(axis=1, how='all')
    data_extra =data_extra.drop(columns =['Outlet or Author','Country','Mention Type','External Mention ID'],axis=1)
    print("trim the column")
    print("---Unique Numbers---")
    print(f"Unique DOI counts :{data_extra['DOI'].nunique()}")
    print(f"Unique Wiki Title counts :{data_extra['Mention Title'].nunique()}")
    print(f"Total data counts :{len(data_extra)}")
    
    print("---Check DOI ---")
    DOI_NA = data_extra[data_extra['DOI'].isna()==True ]
    DOI_Count = len(data_extra) - len(DOI_NA)
    #check  PubMedID exist when DOI is empty  
    PubMedID_exist =DOI_NA[DOI_NA["PubMed ID"].isna()==False]
    PubMedID_na =DOI_NA[DOI_NA["PubMed ID"].isna()==True]
    # ArXivID_exist =DOI_NA[DOI_NA["arXiv ID"].isna()==False]
    # ArXivID_na =DOI_NA[DOI_NA["arXiv ID"].isna()==True]
    
    print(f"There are {len(DOI_NA)} empty DOI")
    print(f"In those empty DOI, there are {len(PubMedID_exist)} with PubMedID ")
    print(f"There are {len(PubMedID_na)} withOUT PubMedID ")
    if len(PubMedID_na) > 0:
        print(PubMedID_na )
    
    print(f"Total : {len(data_extra)} data")
    print(" ")
    DOI_NA_table = pd.DataFrame(np.array([[DOI_Count, len(PubMedID_exist)], [len(DOI_NA), len(PubMedID_na)]]),
                       columns=['DOI', 'PubMedID'],index=["counts","empty"])
    print("Print DOIis empty _table")
    print(DOI_NA_table)

    print(" ")
    print("---Checking Affiliations--- ")
    Affil_without_uni = data_extra[~data_extra['Affiliations (GRID)'].str.contains(uni, na=False)]
    print(f"{len(Affil_without_uni)} data , not Uni included in Affiliations ")
    print(f" List the data not include {uni} ")
    print(Affil_without_uni)
    
    #check Mention Date >funder year
    date = data_extra["Mention Date"].str.split(" ",expand=True) 
    year = date[0].str.split("-",expand=True)[0]
    #print(f"Ealiest mention year : {min(year)}")

    #Check Mention Date _ date, format 
    data_extra[['Mention_date','Mention_time']] = data_extra["Mention Date"].str.split(" ",expand=True)  
    try:
        # This will raise an error if any date is not in the correct format
        data_extra['Mention_date'] = pd.to_datetime(data_extra['Mention_date'], format='%Y-%m-%d', errors='raise')
        print("All Mention dates are in the correct format.")
    except ValueError:
        print("Some dates are not in the correct format.",data_extra['Mention_date'])
            
    data_extra =data_extra.drop(columns =['Mention Date'],axis=1)
    
    # Publication Date
    try:
        # This will raise an error if any date is not in the correct format
        data_extra['Publication Date'] = pd.to_datetime(data_extra['Publication Date'], format='%Y-%m-%d', errors='raise')
        print("All Publication dates are in the correct format.")
    except ValueError:
        print("Some dates are not in the correct format.",data_extra['Publication Date'])
    #Mention URL
    print("Any empty URL")
    print(data_extra[data_extra['Mention URL'].isna()== True])

    #Wiki_Language code from url  
    lan = data_extra["Mention URL"]
    lan =lan.str.split("//",expand=True)[1]
    Wiki_Lang_code = lan.str.split(".",expand=True)[0]
    data_extra['Wiki_Language'] = Wiki_Lang_code 
    print(f"Top 10 using Language") #or Top 15
    print(data_extra['Wiki_Language'].value_counts()[:10]) 

    # Altmetric Attention Score
    # # print(data_extra['Altmetric Attention Score'].isna().unique())
    # try:
    #     # This will raise an error if any value is not an integer
    #     data_extra['Altmetric Attention Score'] = pd.to_numeric(data_extra['Altmetric Attention Score'], downcast='integer', errors='raise')
    #     print("All values in the column are integers.")
    # except ValueError:
    #     print("Some values in the column are not integers.")
    #Subjects (FoR) _spilt grid and subject or not))

    #Drop empty DOI
    if len(DOI_NA) > 0:
        data_cleaned =  data_extra.dropna(subset=['DOI'])
        data_cleaned = data_cleaned[data_cleaned['DOI'].str.strip()!='']
        data_cleaned.reset_index(drop=True,inplace=True)
    else :
        data_cleaned = data_extra

    print("Last Cleaned Data")
    # print(data_cleaned)
    return data_cleaned

In [53]:
def uni_group(x) :
    grouping_type = ""
    g1 = ["Glasgow School of Art","Royal Conservatoire of Scotland","Scotland's Rural College"]
    g2 = ["University of Edinburgh","University of Glasgow","University of St Andrews","University of Aberdeen"]
    g3 = ["University of Dundee","Heriot-Watt University","University of Strathclyde","University of Stirling"]
    g4 = ["Abertay University","Glasgow Caledonian University","Edinburgh Napier University","Queen Margaret University",
          "University of the Highlands and Islands","University of the West of Scotland","Robert Gordon University"]
    if x in g1 :
        grouping_type = "Specialist" ;
    elif x in g2 :
        grouping_type = "Ancient" ;
    elif x in g3 :
        grouping_type = "Chartered" ;
    elif x in g4 :
        grouping_type = "Modern" ;
    else :
        grouping_type = f"No Group_{u}"
    return grouping_type

In [54]:
def Unpaywalldata(uni,data_cl):

    #add info
    Uni, Grouping = [],[]
    group_type = uni_group(uni)
    #Unpaywall data
    Is_OA,OA_Status = [], []
    Genre ,title, OA_Locations, License_bestOA = [], [], [], []

    #API _all DOI unpaywall data 
    for j in range(len(data_cl)):
        Uni.append(uni)
        Grouping.append(group_type)
        response_API = requests.get(f"https://api.unpaywall.org/v2/{data_cl['DOI'][j]}?email=unpaywall_01@example.com")
        #print(response_API.status_code)
        if response_API.status_code ==200 :
            Unpaywall_data= response_API.text
            parse_json = json.loads(Unpaywall_data)
            active_case = parse_json
            Is_OA.append(active_case['is_oa'])
            OA_Status.append(active_case['oa_status'])
            Genre.append(active_case['genre'])
            title.append(active_case['title'])
            OA_Locations.append(active_case['oa_locations'])
            if active_case['is_oa'] is True :
                License_bestOA.append(active_case['best_oa_location']['license'])
            else :
                License_bestOA.append('Not OA')
        else :
            Is_OA.append('notFound')
            OA_Status.append(response_API.status_code)
            Genre.append('notFound')
            title.append('notFound')
            OA_Locations.append('notFound')
            License_bestOA.append('notFound')
           
            
    #Keep key data
    #clean and turn to dataframe
    Is_OA = pd.DataFrame(Is_OA, columns = ['OA_status'])
    Uni = pd.DataFrame(Uni , columns = ['University'])
    Grouping = pd.DataFrame(Grouping, columns = ['Group'])
    OA_Status = pd.DataFrame(OA_Status , columns = ['OA_Type'])
    #---more information
    Genre = pd.DataFrame(Genre , columns = ['Genre'])
    title = pd.DataFrame(title , columns = ['Title'])
    OA_Locations = pd.Series(OA_Locations)
    OA_Locations = pd.DataFrame(OA_Locations , columns = ['OA_Locations'])
    License_bestOA = pd.DataFrame(License_bestOA , columns = ['License_bestOA'])
    
    
    OA_data= pd.concat([Is_OA,OA_Status,Genre,title,OA_Locations,License_bestOA,Uni,Grouping],axis = 1)
    #print(OA_data)
    New_data = pd.concat([data_cl,OA_data],axis = 1)
    # print(New_data)
    # # New_data.to_csv(f"{x}_AltOA.csv")
    # New_data.to_csv(f"../Data/Unpaywall/New/n_{uni}_AltOA.csv",index=False)
    print("Finish dowonload")
    
    return  New_data

In [55]:
# Group_s = ["Glasgow School of Art","Royal Conservatoire of Scotland","Scotland's Rural College"]
# Group_M = ["Abertay University","Queen Margaret University","Glasgow Caledonian University","Edinburgh Napier University",
#       "University of the West of Scotland","Robert Gordon University","University of the Highlands and Islands"]

### check unpaywall new data

In [56]:
def unpaywall_df_check(ndf) :
    # print(ndf.info())
    print("Check OA_status")
    print("Not found OA_status")
    if len(ndf[ndf['OA_status'].isna() == False]) ==0 :
        print("There is not empty OA_status")
    else  :
        print(ndf[ndf['OA_status'].isna()== True])
    
    #Check 'OA_status' is either True or False
    print(ndf['OA_status'].unique())
    desired_bool = [False, True]
    error_status = ndf[~ndf["OA_status"].isin(desired_bool)]
    print(f"There are TOTAL : {error_status.shape[0]} Data  cannot find through Unpaywall.")
    print(f"There are {ndf[ndf['OA_status']=='notFound'].shape[0]} Data  cannot find through Unpaywall.")
    #List data entries which "notFound" for further check
    print(ndf[ndf['OA_status']=="notFound"][['DOI','OA_Type']])
    
    #Check 'OA_Type' is only those 5 types ['closed','bronze','green','gold','hybrid']
    print(ndf['OA_Type'].unique())
    desired_Type= ['closed','bronze','green','gold','hybrid']
    error_data = ndf[~ndf["OA_Type"].isin(desired_Type)]
    print(" ")
    #List data entries which not expected OA Type for further check
    #confirm the unexpected OA_status and OA_Type is the same data entry 
    print("error data _list (Doi)")
    print(error_data[["DOI",'OA_status','OA_Type',"Mention Title"]])
    print("------")
    
    #1 Cleaned Data (remove empty DOI/error status entries)
    cl_data = ndf[ndf['OA_status']!= "notFound"]
    print("-- Cleaned Data count --")
    print(f"Total catation : {cl_data.shape[0]}")
    print("University OA_ Status count")
    print(cl_data['OA_status'].value_counts())
    print("University OA_ Type count")
    print(cl_data['OA_Type'].value_counts())
    print("------")
    
    #Explore Duplication 
    full_duplicates = cl_data[cl_data.duplicated(subset=["DOI","Mention Title","Mention URL",'Mention_date','Wiki_Language'],keep=False)]
    print("About truly duplicates")
    print(f"There are {full_duplicates['DOI'].nunique()} unique DOI in Duplicate list ")
    print(f"There are {full_duplicates['Mention Title'].nunique()} unique Wiki Title in Duplicate list ")
    print(f"There are Total : {len(full_duplicates)} data in Duplicate list")
    print(" ")
    
    
    duplicates = cl_data[cl_data.duplicated(subset=["DOI","Mention Title",'Mention_date','Wiki_Language'],keep=False)]
    print("About potential duplicates")
    print(f"There are {duplicates['DOI'].nunique()} unique DOI in  potential Duplicate list ")
    print(f"There are {duplicates['Mention Title'].nunique()} unique Wiki Title in potential Duplicate list ")
    print(f"There are Total : {len(duplicates)} data in potential Duplicate list ")
    
    
    #2 De-truly Duplication Data
    No_du_data = cl_data.drop_duplicates(subset=["DOI","Mention Title","Mention URL",'Mention_date','Wiki_Language'],keep="last")
    No_du_data = No_du_data[No_du_data['OA_status']!= "notFound"]
    print(f"De-truly Duplication Total catation : {No_du_data.shape[0]}")
    #3 De-potential Duplicaion Data
    No_pdu_data = cl_data.drop_duplicates(subset=["DOI","Mention Title",'Mention_date','Wiki_Language'],keep="last")
    No_pdu_data = No_pdu_data[No_pdu_data['OA_status']!= "notFound"]
    print(f"De-potential Duplication Total catation : {No_pdu_data.shape[0]}")
    
    print(" ")
    print("Show all data count and percentage")
    def calculate_counts_and_percentages(data):     
        oaStatus_counts = data['OA_status'].value_counts()
        oaStatus_percentages = data['OA_status'].value_counts(normalize = True)*100
        
        OpenOA = data[data['OA_status']==True]
        OpenOAType_counts = OpenOA ['OA_Type'].value_counts()
        OpenOAType_percentages  = OpenOA ['OA_Type'].value_counts(normalize = True)*100
    
        OA_status_table = pd.DataFrame({
           'count':oaStatus_counts ,
           'percentage':oaStatus_percentages
        })
    
        OA_type_table = pd.DataFrame({
           'count':OpenOAType_counts ,
           'percentage':OpenOAType_percentages
        })
        
        data_countandpercent = pd.concat([OA_status_table,OA_type_table] ,keys=['OA_status','OA_Type'])
        return data_countandpercent
    
    #1+2+3    
    combined_data = pd.concat([calculate_counts_and_percentages(cl_data),calculate_counts_and_percentages(No_du_data),calculate_counts_and_percentages(No_pdu_data)] ,axis=1)
    
    print(combined_data)
    

In [57]:
# unpaywall_df_check(Glasgow_New_data)

In [58]:
Group_s = ["Glasgow School of Art","Royal Conservatoire of Scotland","Scotland's Rural College"]
for uni in Group_s :
    print(f"!!{uni}_Start")
    data_cleaned  = Alt_data_check_clean(uni) 
    New_data = Unpaywalldata(uni,data_cleaned)
    print("+++------------------------")
    print('check unpaywall new data')
    # print( New_data.info())
    unpaywall_df_check(New_data)
    print(f"{uni}_done")
    print("------------------------")
    print(" ")

!!Glasgow School of Art_Start
Read the Glasgow School of Art Alt local data file!
trim the column
---Unique Numbers---
Unique DOI counts :7
Unique Wiki Title counts :12
Total data counts :13
---Check DOI ---
There are 0 empty DOI
In those empty DOI, there are 0 with PubMedID 
There are 0 withOUT PubMedID 
Total : 13 data
 
Print DOIis empty _table
        DOI  PubMedID
counts   13         0
empty     0         0
 
---Checking Affiliations--- 
0 data , not Uni included in Affiliations 
 List the data not include Glasgow School of Art 
Empty DataFrame
Columns: [Mention Date, Mention Title, Mention URL, Research Output Title, Journal/Collection Title, Authors at my Institution, Departments, Output Type, Subjects (FoR), Affiliations (GRID), Publication Date, Altmetric Attention Score, Details Page URL, DOI, ISBN, National Clinical Trial ID, URI, PubMed ID, PubMedCentral ID, Handle.net IDs, ADS Bibcode, arXiv ID, RePEc ID, SSRN, URN, Publisher Names]
Index: []

[0 rows x 26 columns]
All Men

In [59]:
Group_M = ["Abertay University","Queen Margaret University","Glasgow Caledonian University","Edinburgh Napier University",
      "University of the West of Scotland","Robert Gordon University","University of the Highlands and Islands"]
for uni in Group_M :
    print(f"!!{uni}_Start")
    data_cleaned  = Alt_data_check_clean(uni) 
    New_data = Unpaywalldata(uni,data_cleaned)
    print("+++------------------------")
    print('check unpaywall new data')
    # print( New_data.info())
    unpaywall_df_check(New_data)
    print(f"{uni}_done")
    print("------------------------")
    print(" ")

!!Abertay University_Start
Read the Abertay University Alt local data file!
trim the column
---Unique Numbers---
Unique DOI counts :94
Unique Wiki Title counts :217
Total data counts :275
---Check DOI ---
There are 0 empty DOI
In those empty DOI, there are 0 with PubMedID 
There are 0 withOUT PubMedID 
Total : 275 data
 
Print DOIis empty _table
        DOI  PubMedID
counts  275         0
empty     0         0
 
---Checking Affiliations--- 
0 data , not Uni included in Affiliations 
 List the data not include Abertay University 
Empty DataFrame
Columns: [Mention Date, Mention Title, Mention URL, Research Output Title, Journal/Collection Title, Authors at my Institution, Departments, Output Type, Subjects (FoR), Affiliations (GRID), Publication Date, Altmetric Attention Score, Details Page URL, DOI, ISBN, National Clinical Trial ID, URI, PubMed ID, PubMedCentral ID, Handle.net IDs, ADS Bibcode, arXiv ID, RePEc ID, SSRN, URN, Publisher Names]
Index: []

[0 rows x 26 columns]
All Mention 

# More

---


In [60]:
# # plt.plot(New_data["Mention_date"], label='All ' ,linestyle = 'dashed')
# plt.hist(Stirling_New_data["Mention_date"], alpha=0.5, label='All Patients', color='grey')
#     # Legend
# plt.legend(loc=0)

# # titles and labels
# plt.title(f'wikiOpencite : "Mention_date"')
# # plt.xlabel(category)
# plt.ylabel('Frequency')